# Data Collection

In [1]:
import json
import praw
import random
from datetime import datetime, timedelta

import string
import codecs
import re
import praw
import numpy as np
import math
from nltk.stem import WordNetLemmatizer

from collections import Counter
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon') 
nltk.download('wordnet')

from colorama import Fore, Back, Style
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.lda_model
from wordcloud import WordCloud

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Chelsea/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Chelsea/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
def redditClient():
    """
        Setup Reedit API authentication.
        Replace username, secrets and passwords with your own.

        @returns: praw Reedit object
    """

    try:
        clientId = "wxXd0m1BJeyk8e063RmgIQ"
        clientSecret = "H5Tup-uhMHVudsDO38hpvXfLn3vbFQ"
        password = "Chelsea0220"
        userName = "Ok_Shopping_473"
        userAgents = 'client for SNAM2024'

        redditClient = praw.Reddit(client_id = clientId,
                                   client_secret = clientSecret,
                                   user_agent = userAgents)
    except KeyError:
        sys.stderr.write("Key or secret token are invalid.\n")
        sys.exit(1)

    return redditClient

In [27]:
def fetch_posts(keywords, limit=5000, filename='posts.json'):
    reddit = redditClient()
    all_posts = []

    for keyword in keywords:
        try:
            # Search for submissions containing the keyword
            submissions = reddit.subreddit('all').search(keyword, limit=limit)

            posts = []
            for submission in submissions:
                submission_data = {
                    'id': submission.id,
                    'title': submission.title,
                    'selftext': submission.selftext,
                    'score': submission.score,
                    'created': submission.created_utc,
                    'author_id': submission.author.name if submission.author else 'N/A',
                    'comments': [],
                    'keyword': keyword  # Store the keyword used for the search
                }

                submission.comments.replace_more(limit=0)
                for comment in submission.comments.list():
                    comment_data = {
                        'id': comment.id,
                        'body': comment.body,
                        'score': comment.score,
                        'created': comment.created_utc,
                        'author_id': comment.author.name if comment.author else 'N/A',
                    }
                    submission_data['comments'].append(comment_data)

                posts.append(submission_data)

            print(f"Number of submissions found for keyword '{keyword}': {len(posts)}")
            all_posts.extend(posts)

        except Exception as e:
            print(f"Error fetching posts for keyword '{keyword}': {e}")

    with open(filename, 'w') as file:
        json.dump(all_posts, file, indent=5)

# Usage example with keywords
keywords = ['Olympics 2024', 'Olympics 2020', 'Olympics 2016']
fetch_posts(keywords, limit=5000, filename='olympics_posts.json')

Number of submissions found for keyword 'Olympics 2024': 245
Number of submissions found for keyword 'Olympics 2020': 228
Number of submissions found for keyword 'Olympics 2016': 231


# Data Analysis

In [29]:
# load json file
fJsonName = 'olympics_posts.json'

post_data =[]
# open json file and process it tweet by tweet
with open(fJsonName, 'r') as f:
    dSubmissions = json.load(f)
    
    for submission in dSubmissions[1:]:
        submissionID = submission['id']
        submissionsTitle = submission['title']
        submissionSelftext = submission['selftext']
        submissionCreated = submission['created']
        submissionAuthor = submission['author_id']

        # store all process data
        post_data.append({
            'ID': submissionID,
            'Title': submissionsTitle,
            'Selftext': submissionSelftext,
            'Created': submissionCreated
        })

# Convert a DataFrame
df_post = pd.DataFrame(post_data)
# change datetime
df_post['Created'] = pd.to_numeric(df_post['Created'], errors='coerce')
df_post['Created'] = pd.to_datetime(df_post['Created'], unit='s')

print(df_post)

          ID                                              Title  \
0    1ef4lku  USA, a country so obsessed with guns, has so f...   
1    1eeaa5m          Most controversial pic from olympics 2024   
2    1f5ez3o     Paralympic breakdancing at PARIS Olympics 2024   
3    1epqvq1  USA and China tie for most gold medals in the ...   
4    1ej4nz8  The kiss between gold medal's winner Alice Bel...   
..       ...                                                ...   
698   lbvzaf  What's your craziest ride on a skates? Here is...   
699   3phyki  Banco Bradesco, 2016 Olympic Sponsor, destroyi...   
700   5677r2  All 36 Rio 2016 Olympic boxing referees and ju...   
701   4xksxz  Post Match: Nadal vs Del Potro (Semifinal, 201...   
702   viebgm  [Stein] There are credible rumblings in circul...   

                                              Selftext             Created  
0                                                      2024-07-29 16:56:29  
1                                        

In [31]:
# Extract comments
comments_data = []
for post in dSubmissions:
    submission_id = post['id']
    for comment in post['comments']:
        # Include submission ID to link comments to their posts
        comments_data.append({
            'submissionID': submission_id,
            'commentID': comment['id'],
            'commentBody': comment['body'],
            'Created': comment['created'],
            'Author': comment['author_id'],
        })
# Convert a DataFrame
df_comments = pd.DataFrame(comments_data)
df_comments['Created'] = pd.to_numeric(df_comments['Created'], errors='coerce')
df_comments['Created'] = pd.to_datetime(df_comments['Created'], unit='s')
print(df_comments)

       submissionID commentID  \
0           1er8uik   lhxbug8   
1           1er8uik   lhx5ejr   
2           1er8uik   lhy3e0l   
3           1er8uik   lhx9nqm   
4           1er8uik   lhx0zgh   
...             ...       ...   
200611       viebgm   iddjjz7   
200612       viebgm   iddne1h   
200613       viebgm   iddiwjz   
200614       viebgm   ide82w7   
200615       viebgm   iddptq5   

                                              commentBody             Created  \
0       ![gif](giphy|1KHBPmEOkv0B2)\n\nPOV: top right ... 2024-08-13 15:41:17   
1       I just want to know what's in the top corner o... 2024-08-13 15:07:45   
2       I cant believe that this person became\nthe fo... 2024-08-13 18:05:09   
3       Media is more upset about this than the fact t... 2024-08-13 15:29:55   
4       Nobody said anything when she lost in the quar... 2024-08-13 14:44:16   
...                                                   ...                 ...   
200611       Well said, I agree with